## Upload vegas information into database to match game IDs from NBA API

In [3]:
import nba_api.stats.endpoints as nba
import mysql.connector
from mysql.connector import errorcode
import config
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline



In [79]:
dbname = 'nba'
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    db = dbname
)
cursor = cnx.cursor(buffered = True)

In [9]:
stmt = """SELECT * FROM nba.game_logs"""
cursor.execute(stmt)
games = cursor.fetchall()

In [15]:
field_names = [i[0] for i in cursor.description]
games = pd.DataFrame(game_ids)
games.columns = field_names

In [16]:
games

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,pk
0,22017,1610612738,BOS,Boston Celtics,0021700001,2017-10-17,BOS @ CLE,L,240,36,...,46,24,11,4,12,24,99,-3,1,0021700001BOS
1,22017,1610612739,CLE,Cleveland Cavaliers,0021700001,2017-10-17,CLE vs. BOS,W,240,38,...,50,19,3,4,17,25,102,3,1,0021700001CLE
2,22017,1610612744,GSW,Golden State Warriors,0021700002,2017-10-17,GSW vs. HOU,L,240,43,...,41,34,5,9,17,25,121,-1,1,0021700002GSW
3,22017,1610612745,HOU,Houston Rockets,0021700002,2017-10-17,HOU @ GSW,W,240,47,...,43,28,9,5,13,16,122,1,1,0021700002HOU
4,22017,1610612766,CHA,Charlotte Hornets,0021700003,2017-10-18,CHA @ DET,L,240,29,...,47,16,4,3,17,15,90,-12,1,0021700003CHA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5393,22019,1610612764,WAS,Washington Wizards,0021900237,2019-11-24,WAS vs. SAC,L,240,42,...,45,29,5,2,17,23,106,-7,1,0021900237WAS
5394,22019,1610612743,DEN,Denver Nuggets,0021900238,2019-11-24,DEN vs. PHX,W,240,43,...,52,24,4,2,11,16,116,12,1,0021900238DEN
5395,22019,1610612756,PHX,Phoenix Suns,0021900238,2019-11-24,PHX @ DEN,L,240,41,...,36,25,4,2,12,18,104,-12,1,0021900238PHX
5396,22019,1610612746,LAC,LA Clippers,0021900239,2019-11-24,LAC vs. NOP,W,240,46,...,56,27,4,5,8,21,134,25,1,0021900239LAC


In [85]:
vegas = pd.read_csv('vegas_info.csv', index_col=0)

In [86]:
vegas['team'].unique()

array(['Cavaliers', 'Celtics', 'Warriors', 'Rockets', 'Bucks',
       'Grizzlies', 'Pelicans', 'Jazz', 'Nuggets', 'Kings', 'Magic',
       'Heat', 'Mavericks', 'Hawks', 'Pacers', 'Nets', 'Pistons',
       'Hornets', 'Spurs', 'Timberwolves', 'Suns', 'Trailblazers',
       'Wizards', 'Seventysixers', 'Lakers', 'Clippers', 'Raptors',
       'Bulls', 'Thunder', 'Knicks'], dtype=object)

### Need to join tables to get game id in vegas dataframe by matching games first on date, then by team abbreviation

In [31]:
games.TEAM_ABBREVIATION.unique()

array(['BOS', 'CLE', 'GSW', 'HOU', 'CHA', 'DET', 'BKN', 'IND', 'MIA',
       'ORL', 'PHI', 'WAS', 'MIL', 'MEM', 'NOP', 'ATL', 'DAL', 'DEN',
       'UTA', 'MIN', 'SAS', 'PHX', 'POR', 'SAC', 'CHI', 'TOR', 'NYK',
       'OKC', 'LAC', 'LAL'], dtype=object)

In [34]:
### Create map for team names in vegas df to convert them to team abbreviations used by the NBA
team_map = {'Cavaliers': 'CLE', 
            'Celtics': 'BOS', 
            'Warriors': 'GSW', 
            'Rockets': 'HOU', 
            'Bucks': 'MIL',
            'Grizzlies': 'MEM', 
            'Pelicans': 'NOP', 
            'Jazz': 'UTA', 
            'Nuggets': 'DEN', 
            'Kings': 'SAC', 
            'Magic': 'ORL',
            'Heat': 'MIA', 
            'Mavericks': 'DAL', 
            'Hawks': 'ATL', 
            'Pacers':'IND', 
            'Nets': 'BKN', 
            'Pistons': 'DET',
            'Hornets': 'CHA', 
            'Spurs': 'SAS', 
            'Timberwolves': 'MIN', 
            'Suns': 'PHX', 
            'Trailblazers': 'POR',
            'Wizards': 'WAS', 
            'Seventysixers': 'PHI', 
            'Lakers': 'LAL', 
            'Clippers': 'LAC', 
            'Raptors': 'TOR',
            'Bulls': 'CHI', 
            'Thunder': 'OKC', 
            'Knicks' : 'NYK'
    
}

In [35]:
vegas['team_abbreviation'] = vegas['team'].map(lambda x : team_map[x])

In [40]:
vegas['opposing_team_abbreviation'] = vegas['o:team'].map(lambda x : team_map[x])

In [41]:
vegas

,date,team,site,o:team,line,total,team_abbreviation,opposing_team_abbreviation
0,20171017,Cavaliers,home,Celtics,-3.5,212.5,CLE,BOS
1,20171017,Celtics,away,Cavaliers,3.5,212.5,BOS,CLE
2,20171017,Warriors,home,Rockets,-9.0,228.5,GSW,HOU
3,20171017,Rockets,away,Warriors,9.0,228.5,HOU,GSW
4,20171018,Celtics,home,Bucks,-2.0,206.0,BOS,MIL
...,...,...,...,...,...,...,...,...
7703,20200415,Knicks,away,Timberwolves,-,-,NYK,MIN
7704,20200415,Trailblazers,home,Clippers,-,-,POR,LAC
7705,20200415,Clippers,away,Trailblazers,-,-,LAC,POR
7706,20200415,Wizards,home,Pacers,-,-,WAS,IND


### Manipulate the data to get it in the same format as the game logs table

In [48]:
def convert_date(date):
    return date[:4] + '-' + date[4:6] + '-'+ date[6:]

In [45]:
vegas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7708 entries, 0 to 7707
Data columns (total 8 columns):
date                          7708 non-null int64
team                          7708 non-null object
site                          7708 non-null object
o:team                        7708 non-null object
line                          7708 non-null object
total                         7708 non-null object
team_abbreviation             7708 non-null object
opposing_team_abbreviation    7708 non-null object
dtypes: int64(1), object(7)
memory usage: 542.0+ KB


In [46]:
vegas.date = vegas.date.astype('str')
vegas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7708 entries, 0 to 7707
Data columns (total 8 columns):
date                          7708 non-null object
team                          7708 non-null object
site                          7708 non-null object
o:team                        7708 non-null object
line                          7708 non-null object
total                         7708 non-null object
team_abbreviation             7708 non-null object
opposing_team_abbreviation    7708 non-null object
dtypes: object(8)
memory usage: 542.0+ KB


In [50]:
vegas['match_date'] = vegas['date'].apply(lambda x : convert_date(x))

In [51]:
vegas

,date,team,site,o:team,line,total,team_abbreviation,opposing_team_abbreviation,match_date
0,20171017,Cavaliers,home,Celtics,-3.5,212.5,CLE,BOS,2017-10-17
1,20171017,Celtics,away,Cavaliers,3.5,212.5,BOS,CLE,2017-10-17
2,20171017,Warriors,home,Rockets,-9.0,228.5,GSW,HOU,2017-10-17
3,20171017,Rockets,away,Warriors,9.0,228.5,HOU,GSW,2017-10-17
4,20171018,Celtics,home,Bucks,-2.0,206.0,BOS,MIL,2017-10-18
...,...,...,...,...,...,...,...,...,...
7703,20200415,Knicks,away,Timberwolves,-,-,NYK,MIN,2020-04-15
7704,20200415,Trailblazers,home,Clippers,-,-,POR,LAC,2020-04-15
7705,20200415,Clippers,away,Trailblazers,-,-,LAC,POR,2020-04-15
7706,20200415,Wizards,home,Pacers,-,-,WAS,IND,2020-04-15


In [59]:
games_sub = games[['TEAM_ABBREVIATION', 'GAME_DATE', 'GAME_ID']]

In [60]:
games_sub

,TEAM_ABBREVIATION,GAME_DATE,GAME_ID
0,BOS,2017-10-17,0021700001
1,CLE,2017-10-17,0021700001
2,GSW,2017-10-17,0021700002
3,HOU,2017-10-17,0021700002
4,CHA,2017-10-18,0021700003
...,...,...,...
5393,WAS,2019-11-24,0021900237
5394,DEN,2019-11-24,0021900238
5395,PHX,2019-11-24,0021900238
5396,LAC,2019-11-24,0021900239


In [63]:
combined = pd.merge(left=vegas, right=games_sub, left_on=['team_abbreviation', 'match_date'], right_on=['TEAM_ABBREVIATION', 'GAME_DATE'])

In [64]:
combined.columns

Index(['date', 'team', 'site', 'o:team', 'line', 'total', 'team_abbreviation',
       'opposing_team_abbreviation', 'match_date', 'TEAM_ABBREVIATION',
       'GAME_DATE', 'GAME_ID'],
      dtype='object')

In [65]:
cols = ['match_date', 'team_abbreviation', 'site', 'opposing_team_abbreviation', 'line', 'total', 'GAME_ID']

In [69]:
upload_df = combined[cols]
upload_df.columns

Index(['match_date', 'team_abbreviation', 'site', 'opposing_team_abbreviation',
       'line', 'total', 'GAME_ID'],
      dtype='object')

In [71]:
cols =['date', 'team', 'site', 'oteam', 'line', 'total', 'game_id']
upload_df.columns = cols
upload_df

,date,team,site,oteam,line,total,game_id
0,2017-10-17,CLE,home,BOS,-3.5,212.5,0021700001
1,2017-10-17,BOS,away,CLE,3.5,212.5,0021700001
2,2017-10-17,GSW,home,HOU,-9.0,228.5,0021700002
3,2017-10-17,HOU,away,GSW,9.0,228.5,0021700002
4,2017-10-18,BOS,home,MIL,-2.0,206.0,0021700007
...,...,...,...,...,...,...,...
5393,2019-11-24,PHX,away,DEN,11.0,214.5,0021900238
5394,2019-11-24,HOU,home,DAL,-5.5,232.0,0021900235
5395,2019-11-24,DAL,away,HOU,5.5,232.0,0021900235
5396,2019-11-24,WAS,home,SAC,-2.5,233.0,0021900237


In [73]:
upload_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5398 entries, 0 to 5397
Data columns (total 7 columns):
date       5398 non-null object
team       5398 non-null object
site       5398 non-null object
oteam      5398 non-null object
line       5398 non-null object
total      5398 non-null object
game_id    5398 non-null object
dtypes: object(7)
memory usage: 337.4+ KB


In [76]:
upload_df.line = upload_df.line.astype('float')
upload_df.total = upload_df.total.astype('float')


/Users/marshall132/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [77]:
upload_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5398 entries, 0 to 5397
Data columns (total 7 columns):
date       5398 non-null object
team       5398 non-null object
site       5398 non-null object
oteam      5398 non-null object
line       5398 non-null float64
total      5398 non-null float64
game_id    5398 non-null object
dtypes: float64(2), object(5)
memory usage: 337.4+ KB


In [78]:
tables = {}
tables['vegas'] = (
    "CREATE TABLE vegas ("
    " date varchar(100),"
    " team varchar(3),"
    " site varchar(4),"
    " oteam varchar(3),"
    " line float,"
    " total float,"
    " game_id varchar(100)"
    ") ENGINE=InnoDB")

In [80]:
for table_name in tables:
    table_description = tables[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

Creating table vegas: OK


In [81]:
def insert_into_table(table, df):
    import numpy as np
    
    ### Get column names from table
    stmt = "SELECT * from nba.{}".format(table)
    cursor.execute(stmt)
    scores = cursor.fetchall()
    
    field_names = [i[0] for i in cursor.description]
    
    reordered_df = df[field_names]
    ## replace nan with none for SQL
    reordered_df.replace({np.nan: None}, inplace = True)
    
    ## build insert statement using properly ordered columns
    cols_insert = ''
    str_insert = ''
    for col in reordered_df.columns:
        cols_insert += col + ', '
        str_insert += '%s, '
    cols_insert = cols_insert.strip(', ')
    str_insert = str_insert.strip(', ')
    
    ## extract values from dataframe as tuples
    
    stats = [tuple(row) for row in reordered_df.values]
    
    # Insert and commit all the player stats
    insert_statement = """INSERT INTO nba.{} ({}) VALUES ({})""".format(table, cols_insert, str_insert)
    cursor.executemany(insert_statement, stats)
    cnx.commit()

In [82]:
upload_df

,date,team,site,oteam,line,total,game_id
0,2017-10-17,CLE,home,BOS,-3.5,212.5,0021700001
1,2017-10-17,BOS,away,CLE,3.5,212.5,0021700001
2,2017-10-17,GSW,home,HOU,-9.0,228.5,0021700002
3,2017-10-17,HOU,away,GSW,9.0,228.5,0021700002
4,2017-10-18,BOS,home,MIL,-2.0,206.0,0021700007
...,...,...,...,...,...,...,...
5393,2019-11-24,PHX,away,DEN,11.0,214.5,0021900238
5394,2019-11-24,HOU,home,DAL,-5.5,232.0,0021900235
5395,2019-11-24,DAL,away,HOU,5.5,232.0,0021900235
5396,2019-11-24,WAS,home,SAC,-2.5,233.0,0021900237


In [83]:
insert_into_table('vegas', upload_df)